In [2]:
import pandas as pd
import os
import numpy as np
DATA_PATH = '../01.Data/'
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
def porc_na(df):
    df_na = pd.DataFrame({'Variable'      : df.columns.values,
                          'Missing (%)'     : np.round(100 * df.isna().sum().values / df.shape[0] ,3)})
    return df_na

In [3]:
%%time
food_dataset = pd.read_csv(os.path.join(DATA_PATH,'en.openfoodfacts.org.products.tsv'), encoding='utf-8', sep='\t').reset_index(drop = True)
food_dataset = food_dataset[(food_dataset['nutrition-score-fr_100g'].isna()==0)]#.reset_index(drop = True)
#food_dataset['new_target'] = food_dataset['nutrition-score-uk_100g']
#food_dataset.loc[food_dataset['new_target'].isna(),'new_target'] = 

df_train     = pd.read_csv(os.path.join(DATA_PATH,'food_train.tsv'), index_col='Index', encoding='utf-8', sep='\t').reset_index(drop = True)
df_test      = pd.read_csv(os.path.join(DATA_PATH,'food_X_test.tsv'), index_col='Index', encoding='utf-8', sep='\t').reset_index()
print(food_dataset.shape)
print(df_train.shape)
print(df_test.shape)
print(df_train.shape[0]+df_test.shape[0] )


<string>:2: DtypeWarning: Columns (0,3,5,19,20,24,25,26,27,28,36,37,38,39,48) have mixed types.Specify dtype option on import or set low_memory=False.


(254856, 163)
(102028, 58)
(25508, 58)
127536
CPU times: user 14.3 s, sys: 1.76 s, total: 16 s
Wall time: 16 s


In [4]:
variables = ['creator', 'created_t', 'created_datetime', 'last_modified_t',
       'last_modified_datetime', 'product_name', 'generic_name', 'quantity',
       'packaging', 'packaging_tags', 'brands', 'brands_tags', 'categories',
       'categories_tags', 'categories_en', 'origins', 'origins_tags',
       'manufacturing_places', 'manufacturing_places_tags', 'labels',
       'labels_tags', 'labels_en', 'emb_codes', 'emb_codes_tags',
       'first_packaging_code_geo', 'cities', 'cities_tags', 'purchase_places',
       'stores', 'countries', 'countries_tags', 'countries_en',
       'ingredients_text', 'allergens', 'allergens_en', 'traces',
       'traces_tags', 'traces_en', 'serving_size', 'no_nutriments',
       'additives_n', 'additives', 'additives_tags', 'additives_en',
       'ingredients_from_palm_oil_n', 'ingredients_from_palm_oil',
       'ingredients_from_palm_oil_tags',
       'ingredients_that_may_be_from_palm_oil_n',
       'ingredients_that_may_be_from_palm_oil',
       'ingredients_that_may_be_from_palm_oil_tags', 'pnns_groups_1',
       'pnns_groups_2', 'states', 'states_tags', 'states_en', 'main_category',
       'main_category_en']

In [5]:
def generate_unique(df):
    suma = '_'
    for i in variables:
        suma = suma + df[i].astype(str).fillna('VACIO')
    df['unique_col'] = suma

In [6]:
generate_unique(food_dataset)
generate_unique(df_train)
generate_unique(df_test)

In [9]:
%%time
df_train_new = df_train.merge(food_dataset[['nutrition-score-fr_100g','unique_col']],how = 'left',on = ['unique_col']) 
df_test_new  = df_train_new.drop_duplicates()
df_test_new  = df_test.merge(food_dataset[['nutrition-score-fr_100g','unique_col']],how = 'left',on = ['unique_col']) 
df_test_new  = df_test_new.drop_duplicates(subset = ['Index'],keep = 'first')
df_test_new.shape

CPU times: user 2.87 s, sys: 568 ms, total: 3.43 s
Wall time: 3.43 s


(25508, 60)

In [10]:
df_test_new[['Index','nutrition-score-fr_100g']].rename(columns = {'nutrition-score-fr_100g':'Target'}).to_csv(os.path.join(DATA_PATH,'target_test.csv'),index = False)

In [9]:
from sklearn.metrics import mean_squared_error
mean_squared_error(df_train_new['target'],df_train_new['nutrition-score-fr_100g'])

0.0008105627063028575